# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fed40519c10>
├── 'a' --> tensor([[-0.5337, -1.2711, -0.8742],
│                   [-2.1289,  0.7022,  0.4937]])
└── 'x' --> <FastTreeValue 0x7fed40579f70>
    └── 'c' --> tensor([[-1.0691, -0.4151, -0.1068,  0.3666],
                        [ 1.5807,  1.1638, -1.0991, -0.3018],
                        [-0.8781, -1.1121,  0.7435, -0.6257]])

In [4]:
t.a

tensor([[-0.5337, -1.2711, -0.8742],
        [-2.1289,  0.7022,  0.4937]])

In [5]:
%timeit t.a

58.2 ns ± 0.0814 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fed40519c10>
├── 'a' --> tensor([[ 1.1419, -3.1555, -0.3321],
│                   [ 0.4580, -0.3285,  0.5650]])
└── 'x' --> <FastTreeValue 0x7fed40579f70>
    └── 'c' --> tensor([[-1.0691, -0.4151, -0.1068,  0.3666],
                        [ 1.5807,  1.1638, -1.0991, -0.3018],
                        [-0.8781, -1.1121,  0.7435, -0.6257]])

In [7]:
%timeit t.a = new_value

58.4 ns ± 0.0258 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.5337, -1.2711, -0.8742],
               [-2.1289,  0.7022,  0.4937]]),
    x: Batch(
           c: tensor([[-1.0691, -0.4151, -0.1068,  0.3666],
                      [ 1.5807,  1.1638, -1.0991, -0.3018],
                      [-0.8781, -1.1121,  0.7435, -0.6257]]),
       ),
)

In [10]:
b.a

tensor([[-0.5337, -1.2711, -0.8742],
        [-2.1289,  0.7022,  0.4937]])

In [11]:
%timeit b.a

51.2 ns ± 0.045 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.9679,  0.6757,  0.5367],
               [ 0.3844, -1.0270,  1.6658]]),
    x: Batch(
           c: tensor([[-1.0691, -0.4151, -0.1068,  0.3666],
                      [ 1.5807,  1.1638, -1.0991, -0.3018],
                      [-0.8781, -1.1121,  0.7435, -0.6257]]),
       ),
)

In [13]:
%timeit b.a = new_value

488 ns ± 0.179 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

836 ns ± 0.252 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 37.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

140 µs ± 256 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 220 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fed40514130>
├── 'a' --> tensor([[[-0.5337, -1.2711, -0.8742],
│                    [-2.1289,  0.7022,  0.4937]],
│           
│                   [[-0.5337, -1.2711, -0.8742],
│                    [-2.1289,  0.7022,  0.4937]],
│           
│                   [[-0.5337, -1.2711, -0.8742],
│                    [-2.1289,  0.7022,  0.4937]],
│           
│                   [[-0.5337, -1.2711, -0.8742],
│                    [-2.1289,  0.7022,  0.4937]],
│           
│                   [[-0.5337, -1.2711, -0.8742],
│                    [-2.1289,  0.7022,  0.4937]],
│           
│                   [[-0.5337, -1.2711, -0.8742],
│                    [-2.1289,  0.7022,  0.4937]],
│           
│                   [[-0.5337, -1.2711, -0.8742],
│                    [-2.1289,  0.7022,  0.4937]],
│           
│                   [[-0.5337, -1.2711, -0.8742],
│                    [-2.1289,  0.7022,  0.4937]]])
└── 'x' --> <FastTreeValue 0x7fec903eabb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 124 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fec89dbdc40>
├── 'a' --> tensor([[-0.5337, -1.2711, -0.8742],
│                   [-2.1289,  0.7022,  0.4937],
│                   [-0.5337, -1.2711, -0.8742],
│                   [-2.1289,  0.7022,  0.4937],
│                   [-0.5337, -1.2711, -0.8742],
│                   [-2.1289,  0.7022,  0.4937],
│                   [-0.5337, -1.2711, -0.8742],
│                   [-2.1289,  0.7022,  0.4937],
│                   [-0.5337, -1.2711, -0.8742],
│                   [-2.1289,  0.7022,  0.4937],
│                   [-0.5337, -1.2711, -0.8742],
│                   [-2.1289,  0.7022,  0.4937],
│                   [-0.5337, -1.2711, -0.8742],
│                   [-2.1289,  0.7022,  0.4937],
│                   [-0.5337, -1.2711, -0.8742],
│                   [-2.1289,  0.7022,  0.4937]])
└── 'x' --> <FastTreeValue 0x7fec897f17c0>
    └── 'c' --> tensor([[-1.0691, -0.4151, -0.1068,  0.3666],
                        [ 1.5807,  1.1638, -1.0991, -0.3018],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 127 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.1 µs ± 150 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.0691, -0.4151, -0.1068,  0.3666],
                       [ 1.5807,  1.1638, -1.0991, -0.3018],
                       [-0.8781, -1.1121,  0.7435, -0.6257]],
              
                      [[-1.0691, -0.4151, -0.1068,  0.3666],
                       [ 1.5807,  1.1638, -1.0991, -0.3018],
                       [-0.8781, -1.1121,  0.7435, -0.6257]],
              
                      [[-1.0691, -0.4151, -0.1068,  0.3666],
                       [ 1.5807,  1.1638, -1.0991, -0.3018],
                       [-0.8781, -1.1121,  0.7435, -0.6257]],
              
                      [[-1.0691, -0.4151, -0.1068,  0.3666],
                       [ 1.5807,  1.1638, -1.0991, -0.3018],
                       [-0.8781, -1.1121,  0.7435, -0.6257]],
              
                      [[-1.0691, -0.4151, -0.1068,  0.3666],
                       [ 1.5807,  1.1638, -1.0991, -0.3018],
                       [-0.8781, -1.1121,  0.7435, -0.6257]],

In [26]:
%timeit Batch.stack(batches)

78.2 µs ± 133 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.0691, -0.4151, -0.1068,  0.3666],
                      [ 1.5807,  1.1638, -1.0991, -0.3018],
                      [-0.8781, -1.1121,  0.7435, -0.6257],
                      [-1.0691, -0.4151, -0.1068,  0.3666],
                      [ 1.5807,  1.1638, -1.0991, -0.3018],
                      [-0.8781, -1.1121,  0.7435, -0.6257],
                      [-1.0691, -0.4151, -0.1068,  0.3666],
                      [ 1.5807,  1.1638, -1.0991, -0.3018],
                      [-0.8781, -1.1121,  0.7435, -0.6257],
                      [-1.0691, -0.4151, -0.1068,  0.3666],
                      [ 1.5807,  1.1638, -1.0991, -0.3018],
                      [-0.8781, -1.1121,  0.7435, -0.6257],
                      [-1.0691, -0.4151, -0.1068,  0.3666],
                      [ 1.5807,  1.1638, -1.0991, -0.3018],
                      [-0.8781, -1.1121,  0.7435, -0.6257],
                      [-1.0691, -0.4151, -0.1068,  0.3666],
                   

In [28]:
%timeit Batch.cat(batches)

162 µs ± 310 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

315 µs ± 3.01 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
